# Dependencies

In [1]:
import numpy as np
import scipy as sp
import seaborn as sn
import pandas as pd
import matplotlib
matplotlib.use('agg')
from matplotlib import pyplot as plt
import matplotlib.animation
import time
from IPython.display import HTML, Image, Video
from tqdm import tqdm
import os
import xarray as xr
import gc
import multiprocessing

from mystatsfunctions import OLSE,LMoments
from moarpalettes import get_palette

matplotlib.rcParams['axes.prop_cycle']=matplotlib.cycler('color',list(get_palette.Petroff10().to_sn_palette()))
matplotlib.rcParams['font.family']='Helvetica'
matplotlib.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**30

# %matplotlib inline

# Introduction
This notebook uses the Lorenz '63 system to demonstrate the conceptual differences between conventional "probabilistic" climate-model based attribution and our forecast-based approach. 

The specific system used is the Palmer '99 variant of the Lorenz dynamical model, which includes a forcing (at an angle $\theta$ in the xy place) to the system, representing some external forcing - eg. anthropogenic greenhouse gas emissions. The equations of this system are as follows:

$
\begin{align}
\dot{x} & = \sigma(y-x) + f_0 \cos\theta \\
\dot{y} & = x (\rho - z) - y + f_0 \sin\theta \\
\dot{z} & = xy - \beta z
\end{align}
$

Lorenz, E. N. (1963). Deterministic Nonperiodic Flow. Journal of the Atmospheric Sciences. [DOI](https://doi.org/10.1175/1520-0469(1963)020<0130:DNF>2.0.CO;2)

Palmer, T. N. (1999). A Nonlinear Dynamical Perspective on Climate Prediction. Journal of Climate, 12(2), 575–591. [DOI](https://doi.org/10.1175/1520-0442(1999)012<0575:ANDPOC>2.0.CO;2)

## Define the system

In [2]:
def lorenz(xyz, t, rho=28, sigma=10, beta=8/3, F=0, theta=0):
    
    "Defines the lorez63 dynamical system. Standard default values."
    
    x, y, z = xyz
    x_dot = sigma * (y - x) + F * np.cos(theta)
    y_dot = x * rho - x * z - y + F * np.sin(theta)
    z_dot = x * y - beta * z
    return [x_dot, y_dot, z_dot]

## Integrate the system for long times
Analogous to running a climate model for many years.

We will use a timestep of 1/100 units & run the system for 100 000 units x 100 runs. Each run will start using the final point of the previous run.

In [3]:
# set solver parameters
# start_time = pd.to_datetime('1750-01-01')
# end_time = pd.to_datetime('2250-01-01')
# timeindex = pd.date_range(start_time,end_time,freq='1h')[:-1]
# runlength = (end_time.to_pydatetime()-start_time.to_pydatetime()).days
# timestep = runlength / timeindex.size
## run over this many ICs to generate branches
branches = 101

# set Lorenz system parameters
s = 10
r = 28
b = 8/3
F = (8,np.deg2rad(-40))

In [5]:
# ## original ICs
# IC0 = [0.01,0,0]
# IC1 = IC0[:]

# ## ICs if restarting mid-integration
# # start_from = 42
# # X = xr.open_dataset('Lorenz63-realisations/00'+str(start_from)+'.nc')
# # IC0 = X.isel(time=0).sel(type='unforced').to_array().values.flatten()
# # IC1 = X.isel(time=0).sel(type='forced').to_array().values.flatten()

# for branch in tqdm(np.arange(branches)[:]):
    
#     runlength = 100000
#     timestep = 0.01
#     start = branch * runlength
#     end = (branch+1) * runlength
#     timeindex = np.arange(start,end,timestep)
    
#     # unforced system
#     X0 = sp.integrate.odeint(lorenz, IC0, np.arange(0,runlength+timestep,timestep), (r,s,b,0,0)).reshape(-1,1,3)
#     IC0 = X0[-1,0,:]
#     X0 = xr.DataArray(data=X0[:-1],dims=['time','branch','dim'],coords=dict(time=timeindex,branch=[branch],dim=['x','y','z'])).to_dataset(dim="dim")

#     # system with external forcing
#     ## define forcing in terms of magnitude + direction
#     X1 = sp.integrate.odeint(lorenz, IC1, np.arange(0,runlength+timestep,timestep), (r,s,b,*F)).reshape(-1,1,3)
#     IC1 = X1[-1,0,:]
#     X1 = xr.DataArray(data=X1[:-1],dims=['time','branch','dim'],coords=dict(time=timeindex,branch=[branch],dim=['x','y','z'])).to_dataset(dim="dim")

#     X = xr.concat([X0.expand_dims({'type':['unforced']}),X1.expand_dims({'type':['forced']})],dim='type')
    
#     X.to_netcdf('./Lorenz63-realisations/'+f"{branch:04d}"+'.nc')

 46%|████▌     | 46/101 [3:32:28<4:14:02, 277.14s/it]


KeyboardInterrupt: 

Compute the diagnostics required from the system. 

We'll use $|x+y|$ as our "impact measure".

Trying to demo that there's a difference between storyline & probabilistic -> can get different answers

Question is the answers from my approach give more reliable guide to full ensemble vs. eg. data assimilation

Limit in which has to be so -> do we converge to that result or not?

One argument we could use: if we back off far enough sampling bigger range of possible forcing impacts as we sample range of states

Moving from state with little forcing impact into region with large forcing impact

Lorenz -> very sensitive to forcing about x,y=0

## Reattempt using |x| as the metric of choice
Note: to make loading data in considerable quicker, don't decode_times in the xarray open call.

In [38]:
def preprocess_to_extremes(ds):
    
    ds = ds.load()
    
    rh_events = ds.groupby('time.year').apply(lambda x: x.sel(time=x.x.idxmax('time')))
    lh_events = ds.groupby('time.year').apply(lambda x: x.sel(time=x.x.idxmin('time')))
    
    return xr.concat([rh_events.expand_dims({'event':['max']}),lh_events.expand_dims({'event':['min']})],'event')

In [39]:
X = xr.open_mfdataset('./Lorenz63-realisations/00*.nc',preprocess=preprocess_to_extremes)

In [64]:
test=xr.open_dataset('./Lorenz63-realisations/0000.nc',decode_times=False)

In [80]:
test.sel(time=test.isel(time=slice(10000,None)).x.idxmax('time'))

<xarray.Dataset>
Dimensions:  (branch: 1, type: 2)
Coordinates:
  * type     (type) object 'unforced' 'forced'
    time     (type, branch) int64 2825812 2651661
  * branch   (branch) int64 0
Data variables:
    x        (type, branch) float64 ...
    y        (type, branch) float64 ...
    z        (type, branch) float64 ...

In [70]:
(np.sqrt((test.sel(type='forced')-test_event).to_array()**2).sum('variable')).min()

<xarray.DataArray ()>
array(1.65350325)
Coordinates:
    type     <U6 'forced'
    event    <U3 'max'
    year     int64 1813

In [26]:
def locate_analogs(X0, n=1000, max_dist=2):
    
    """Finds analogs for X0 along the attractor defined by X."""
    
    X0 = X0.squeeze()
    
    E = xr.ufuncs.sqrt(((X.sel(type=X0.type.values) - X0).to_array()**2).sum('variable'))
    
    analog_times = E.rename('E').to_dataframe().sort_values('E').iloc[:n].index.values
    
    analogs = X.sel(time=analog_times,type=X0.type.values).drop('metric')
    
    return analogs.rename(time='number').assign_coords(number=np.arange(n))

In [27]:
E = locate_analogs(rh_event)

In [28]:
E_ranked = E.rename('E').to_dataframe().sort_values('E')

In [71]:
E_ranked

,type,E
time,,
1926-07-30 19:00:00,forced,0.000000
2210-06-13 07:00:00,forced,0.553556
1993-12-09 16:00:00,forced,0.754235
2141-05-03 00:00:00,forced,0.804809
2044-05-12 09:00:00,forced,1.019194
...,...,...
1817-08-05 12:00:00,forced,61.589837
2047-07-29 16:00:00,forced,61.698691
2221-06-22 23:00:00,forced,61.705505


In [21]:
def generate_forecasts(event,
                       leads = [1/8,1/4,1/2,3/4,1,2,3,5,10,20],
                       ens_size = 51,
                       ICP_type = 'random',
                       ran_gen = sp.stats.norm(0,0.005),
                       fc_timestep_size='15min'):
    
    """
    Creates a set of initalised Lorenz63 runs.
    
    Can choose how IC pertubrations are generated.
    """
    
    # choose leads (current freq "days")
    inidates = event.maxtime.values - pd.to_timedelta(leads,unit='d').round('1h')

    # extract initial conditions before the event
    ICs = X.sel(type='forced',time=inidates).rename(time='inidate').drop('metric')

    # generate perturbed initial conditions
    if ICP_type == 'random':
        
        ICPs = xr.DataArray(data=ran_gen.rvs(ens_size*3).reshape(ens_size,3),dims=['number','variable'],coords=dict(number=np.arange(ens_size),variable=['x','y','z']))
        ## make the first intial condition the "control"
        ICPs[0] = 0
        # add perturbations to control ICs to get array of ICs
        ICs = (ICs.to_array() + ICPs).transpose('inidate','number','variable')
        
    elif ICP_type == 'analog':
        
        ICs = ICs.groupby('inidate').apply(locate_analogs,n=ens_size)
        ICs = ICs.to_array().transpose('inidate','number','variable')
    
    ## integrate each ensemble member until a couple of days after the event
    ## convert timestep size to units of days
    fc_timestep = pd.to_timedelta(fc_timestep_size).total_seconds()/(24*3600)
    fc_end = event.maxtime.values + pd.Timedelta('2d')
    max_fc_length = ((fc_end - inidates).days+(fc_end - inidates).seconds/(3600*24)).max() / fc_timestep

    ### create results array
    fcs0 = np.empty((ens_size,int(max_fc_length),3,inidates.size))+np.nan
    fcs1 = np.empty((ens_size,int(max_fc_length),3,inidates.size))+np.nan

    for i,inidate in enumerate(ICs.inidate.values):

        fc_length = (fc_end - inidate)
        fc_timesteps = int(fc_length.total_seconds()/(fc_timestep*(24*60*60)))
        ics = ICs.sel(inidate=inidate).values

        fc0 = np.array([sp.integrate.odeint(lorenz, ic, np.arange(0,fc_timesteps*fc_timestep-1e-11,fc_timestep), (r,s,b,0,0)) for ic in ics])
        fc1 = np.array([sp.integrate.odeint(lorenz, ic, np.arange(0,fc_timesteps*fc_timestep-1e-11,fc_timestep), (r,s,b,*F)) for ic in ics])

        fcs0[:,-fc_timesteps:,:,i] = fc0
        fcs1[:,-fc_timesteps:,:,i] = fc1

    # wrangle into nice forecast DataArray
    fc_timeindex = pd.date_range(inidates.min(),fc_end,freq=fc_timestep_size)[:-1]
    fcs0 = xr.DataArray(data=fcs0,dims=['number','time','variable','inidate'],coords=dict(number=np.arange(ens_size),time=fc_timeindex,variable=['x','y','z'],inidate=inidates)).to_dataset(dim="variable")
    fcs1 = xr.DataArray(data=fcs1,dims=['number','time','variable','inidate'],coords=dict(number=np.arange(ens_size),time=fc_timeindex,variable=['x','y','z'],inidate=inidates)).to_dataset(dim="variable")
    ## join datasets together
    fcs = xr.concat([fcs0.expand_dims({'type':['unforced']}),fcs1.expand_dims({'type':['forced']})],dim='type')

    # create impact variable
#     fcs['impact'] = xr.ufuncs.fabs(fcs.x+fcs.y)
    
    return fcs

In [22]:
## event I (right hand lobe): '1926-07-30T19:00:00.000000000'
## event II (left hand lobe): '1897-01-17T04:00:00.000000000'
rh_event = X_day.sel(time='1926-07-30',type='forced')
fcs_r = generate_forecasts(rh_event, leads=[1/4,1,2], ens_size=1001, ICP_type='analog', ran_gen=sp.stats.norm(0,1), fc_timestep_size='1min')
fcs_r['metric'] = np.fabs(fcs_r.x)

In [23]:
lh_event = X_day.sel(time='1897-01-17',type='forced')
fcs_l = generate_forecasts(lh_event, leads=[1/4,1,2], ens_size=1001, ICP_type='analog', ran_gen=sp.stats.norm(0,1), fc_timestep_size='1min')
fcs_l['metric'] = np.fabs(fcs_l.x)

In [30]:
"""Choose whether to plot interactively (keep off for generating anomations)"""
%matplotlib inline

In [ ]:
g=sn.displot(data=fcs_l.x.min('time').to_dataframe().reset_index(),x='x',hue='type',col='inidate',kind='hist',element='step',bins=np.linspace(-21,-16,51))
g.fig.patch.set_facecolor('xkcd:white')
# g.fig.savefig('./fcs_l.png',dpi=200)

In [ ]:
g=sn.displot(data=fcs_r.x.max('time').to_dataframe().reset_index(),x='x',hue='type',col='inidate',kind='hist',element='step',bins=np.linspace(16,21,51))
g.fig.patch.set_facecolor('xkcd:white')
# g.fig.savefig('./fcs_r.png',dpi=200)

Animate the left node event first.

In [ ]:
## left node
event_start_time = '1897-01-17T00:00:00'
event_end_time = '1897-01-17T08:00:00'
choose_mems = np.random.choice(500,15)
plot_colors = dict(forced=get_palette.Petroff6().to_sn_palette()[0],unforced=get_palette.Petroff6().to_sn_palette()[1])

def generate_plot(fr_no,inidate):
    
    fig,axes = plt.subplots(1,2,figsize=(10,5))
    ax,ax1=axes
    sn.despine()
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax1.set_xlabel('x')
    ax1.set_xlim(-21,-16)
    ax1.set_ylim(0,150)

    hist0_points = []
    hist1_points = []

    for lnum,mem in enumerate(choose_mems):

        X0 = fcs_l.sel(number=mem,inidate=inidate,type='unforced',time=slice(inidate,event_end_time))
        X1 = fcs_l.sel(number=mem,inidate=inidate,type='forced',time=slice(inidate,event_end_time))

        maxtime0 = X0.x.idxmin('time')
        maxtime1 = X1.x.idxmin('time')

        argtime0 = ((maxtime0 - X0.time.isel(time=0)).astype(float)//(1e9*60)).values[()].astype(int)
        argtime1 = ((maxtime1 - X1.time.isel(time=0)).astype(float)//(1e9*60)).values[()].astype(int)

        X00 = X0.where(X0.time<=maxtime0).isel(time=slice(0,fr_no)).squeeze()
        ax.plot(X00.x,X00.y,lw=1,color=plot_colors['unforced'],zorder=5)

        X10 = X1.where(X1.time<=maxtime1).isel(time=slice(0,fr_no)).squeeze()
        ax.plot(X10.x,X10.y,lw=1,color=plot_colors['forced'],zorder=5)

        if fr_no >= argtime0:
            X01 = X0.where(X0.time>=maxtime0).isel(time=slice(0,fr_no)).squeeze()
            ax.plot(X01.x,X01.y,lw=1,color=plot_colors['unforced'],alpha=0.1,zorder=3)

            X0.sel(time=maxtime0).squeeze().plot.scatter('x','y',color=plot_colors['unforced'],ax=ax,zorder=10,alpha=np.exp(0.02*(argtime0-fr_no)))


        if fr_no >= argtime1:
            X11 = X1.where(X1.time>=maxtime1).isel(time=slice(0,fr_no)).squeeze()
            ax.plot(X11.x,X11.y,lw=1,color=plot_colors['forced'],alpha=0.1,zorder=3)

            X1.sel(time=maxtime1).squeeze().plot.scatter('x','y',color=plot_colors['forced'],ax=ax,zorder=10,alpha=np.exp(0.02*(argtime1-fr_no)))

    ## get histogram points
    if fr_no>0:
        event_max0 = fcs_l.sel(inidate=inidate,type='unforced',time=slice(inidate,event_end_time)).x.min('time').squeeze().values
        actual_max0 = fcs_l.sel(inidate=inidate,type='unforced',time=slice(inidate,event_end_time)).isel(time=slice(0,fr_no)).x.min('time').squeeze().values
        hist0_points = event_max0[event_max0==actual_max0]

        event_max1 = fcs_l.sel(inidate=inidate,type='forced',time=slice(inidate,event_end_time)).x.min('time').squeeze().values
        actual_max1 = fcs_l.sel(inidate=inidate,type='forced',time=slice(inidate,event_end_time)).isel(time=slice(0,fr_no)).x.min('time').squeeze().values
        hist1_points = event_max1[event_max1==actual_max1]

    ## plot ICs
    fcs_l.sel(number=choose_mems,type='forced',inidate=inidate,time=inidate).plot.scatter('x','y',color='k',ax=ax,marker='.')
    fcs_l.sel(type='forced',inidate=inidate,time=inidate).plot.scatter('x','y',color='grey',ax=ax,marker='.',s=1,zorder=-1)

    ## plot histograms
    bins = np.linspace(-21,-16,51)
    ax1.hist(hist0_points, bins=bins, lw=1.5, color=plot_colors['unforced'], histtype='step',label='unforced')
    ax1.hist(hist1_points, bins=bins, lw=1.5, color=plot_colors['forced'], histtype='step',label='forced')

    # figure layout
    ## x,y lim
    xlim0 = fcs_l.sel(number=0,type='forced',inidate=inidate,time=inidate).x.values[()] + np.array([-4,4])
    ylim0 = fcs_l.sel(number=0,type='forced',inidate=inidate,time=inidate).y.values[()] + np.array([-4,4])

    xlim1 = np.array([-25,25])
    ylim1 = np.array([-25,25])

    frames_to_zoom = 120

    curr_xlim = ax.set_xlim()
    curr_ylim = ax.set_ylim()

    xlim_min = np.min([curr_xlim[0],np.max([xlim1[0],xlim0[0]+(fr_no-180)*(xlim1[0]-xlim0[0])/frames_to_zoom])])
    xlim_max = np.max([curr_xlim[1],np.min([xlim1[1],xlim0[1]+(fr_no-180)*(xlim1[1]-xlim0[1])/frames_to_zoom])])

    ylim_min = np.min([curr_ylim[0],np.max([ylim1[0],ylim0[0]+(fr_no-180)*(ylim1[0]-ylim0[0])/frames_to_zoom])])
    ylim_max = np.max([curr_ylim[1],np.min([ylim1[1],ylim0[1]+(fr_no-180)*(ylim1[1]-ylim0[1])/frames_to_zoom])])

    ax.set_xlim(xlim_min,xlim_max)
    ax.set_ylim(ylim_min,ylim_max)
    
    ax1.legend(frameon=False,loc='upper right')

    fig.savefig('../Figures/animation-figs/'+f"{fr_no:04d}"+'_'+str(inidate)+'.png',dpi=200)

    fig.clear()
    plt.close(fig)
    gc.collect()

for inidate in tqdm(fcs_l.inidate.values):

    total_frames = int((pd.to_datetime(event_end_time) - pd.to_datetime(inidate)).total_seconds()/60)

    P1 = multiprocessing.Pool(processes=4)
    
    P1.starmap(generate_plot,[(fr_no,inidate) for fr_no in np.arange(total_frames)])
    
    P1.close()

Then the right.

In [ ]:
## right node
event_start_time = '1926-07-30T15:00'
event_end_time = '1926-07-30T23:00'
choose_mems = np.random.choice(500,15)
plot_colors = dict(forced=get_palette.Petroff6().to_sn_palette()[0],unforced=get_palette.Petroff6().to_sn_palette()[1])

def generate_plot(fr_no,inidate):
    
    fig,axes = plt.subplots(1,2,figsize=(10,5))
    ax,ax1=axes
    sn.despine()
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax1.set_xlabel('x')
    ax1.set_xlim(16,21)
    ax1.set_ylim(0,150)

    hist0_points = []
    hist1_points = []

    for lnum,mem in enumerate(choose_mems):

        X0 = fcs_r.sel(number=mem,inidate=inidate,type='unforced',time=slice(inidate,event_end_time))
        X1 = fcs_r.sel(number=mem,inidate=inidate,type='forced',time=slice(inidate,event_end_time))

        maxtime0 = X0.x.idxmax('time')
        maxtime1 = X1.x.idxmax('time')

        argtime0 = ((maxtime0 - X0.time.isel(time=0)).astype(float)//(1e9*60)).values[()].astype(int)
        argtime1 = ((maxtime1 - X1.time.isel(time=0)).astype(float)//(1e9*60)).values[()].astype(int)

        X00 = X0.where(X0.time<=maxtime0).isel(time=slice(0,fr_no)).squeeze()
        ax.plot(X00.x,X00.y,lw=1,color=plot_colors['unforced'],zorder=5)

        X10 = X1.where(X1.time<=maxtime1).isel(time=slice(0,fr_no)).squeeze()
        ax.plot(X10.x,X10.y,lw=1,color=plot_colors['forced'],zorder=5)

        if fr_no >= argtime0:
            X01 = X0.where(X0.time>=maxtime0).isel(time=slice(0,fr_no)).squeeze()
            ax.plot(X01.x,X01.y,lw=1,color=plot_colors['unforced'],alpha=0.1,zorder=3)

            X0.sel(time=maxtime0).squeeze().plot.scatter('x','y',color=plot_colors['unforced'],ax=ax,zorder=10,alpha=np.exp(0.02*(argtime0-fr_no)))


        if fr_no >= argtime1:
            X11 = X1.where(X1.time>=maxtime1).isel(time=slice(0,fr_no)).squeeze()
            ax.plot(X11.x,X11.y,lw=1,color=plot_colors['forced'],alpha=0.1,zorder=3)

            X1.sel(time=maxtime1).squeeze().plot.scatter('x','y',color=plot_colors['forced'],ax=ax,zorder=10,alpha=np.exp(0.02*(argtime1-fr_no)))

    ## get histogram points
    if fr_no>0:
        event_max0 = fcs_r.sel(inidate=inidate,type='unforced',time=slice(inidate,event_end_time)).x.max('time').squeeze().values
        actual_max0 = fcs_r.sel(inidate=inidate,type='unforced',time=slice(inidate,event_end_time)).isel(time=slice(0,fr_no)).x.max('time').squeeze().values
        hist0_points = event_max0[event_max0==actual_max0]

        event_max1 = fcs_r.sel(inidate=inidate,type='forced',time=slice(inidate,event_end_time)).x.max('time').squeeze().values
        actual_max1 = fcs_r.sel(inidate=inidate,type='forced',time=slice(inidate,event_end_time)).isel(time=slice(0,fr_no)).x.max('time').squeeze().values
        hist1_points = event_max1[event_max1==actual_max1]

    ## plot ICs
    fcs_r.sel(number=choose_mems,type='forced',inidate=inidate,time=inidate).plot.scatter('x','y',color='k',ax=ax,marker='.')
    fcs_r.sel(type='forced',inidate=inidate,time=inidate).plot.scatter('x','y',color='grey',ax=ax,marker='.',s=1,zorder=-1)

    ## plot histograms
    bins = np.linspace(16,21,51)
    ax1.hist(hist0_points, bins=bins, lw=1.5, color=plot_colors['unforced'], histtype='step',label='unforced')
    ax1.hist(hist1_points, bins=bins, lw=1.5, color=plot_colors['forced'], histtype='step',label='forced')

    # figure layout
    ## x,y lim
    xlim0 = fcs_r.sel(number=0,type='forced',inidate=inidate,time=inidate).x.values[()] + np.array([-4,4])
    ylim0 = fcs_r.sel(number=0,type='forced',inidate=inidate,time=inidate).y.values[()] + np.array([-4,4])

    xlim1 = np.array([-25,25])
    ylim1 = np.array([-25,25])

    frames_to_zoom = 120

    curr_xlim = ax.set_xlim()
    curr_ylim = ax.set_ylim()

    xlim_min = np.min([curr_xlim[0],np.max([xlim1[0],xlim0[0]+(fr_no-180)*(xlim1[0]-xlim0[0])/frames_to_zoom])])
    xlim_max = np.max([curr_xlim[1],np.min([xlim1[1],xlim0[1]+(fr_no-180)*(xlim1[1]-xlim0[1])/frames_to_zoom])])

    ylim_min = np.min([curr_ylim[0],np.max([ylim1[0],ylim0[0]+(fr_no-180)*(ylim1[0]-ylim0[0])/frames_to_zoom])])
    ylim_max = np.max([curr_ylim[1],np.min([ylim1[1],ylim0[1]+(fr_no-180)*(ylim1[1]-ylim0[1])/frames_to_zoom])])

    ax.set_xlim(xlim_min,xlim_max)
    ax.set_ylim(ylim_min,ylim_max)
    
    ax1.legend(frameon=False,loc='upper right')

    fig.savefig('../Figures/animation-figs/'+f"{fr_no:04d}"+'_'+str(inidate)+'.png',dpi=200)

    fig.clear()
    plt.close(fig)
    gc.collect()

for inidate in tqdm(fcs_r.inidate.values):

    total_frames = int((pd.to_datetime(event_end_time) - pd.to_datetime(inidate)).total_seconds()/60)
    
    P1 = multiprocessing.Pool(processes=4)
    
    P1.starmap(generate_plot,[(fr_no,inidate) for fr_no in np.arange(total_frames)])
    
    P1.close()

In [ ]:
for inidate in tqdm(fcs_r.inidate.values):

    fig=plt.figure()

    hist0_points = fcs_r.sel(inidate=inidate,type='unforced',time=slice(inidate,event_end_time)).x.max('time').squeeze().values
    hist1_points = fcs_r.sel(inidate=inidate,type='forced',time=slice(inidate,event_end_time)).x.max('time').squeeze().values

    bins = np.linspace(16,21,51)
    plt.hist(hist0_points, bins=bins, lw=1.5, color=plot_colors['unforced'], histtype='step')
    plt.hist(hist1_points, bins=bins, lw=1.5, color=plot_colors['forced'], histtype='step')

    plt.savefig('./test'+str(inidate)+'.png',dpi=200)

    fig.clear()

    plt.close()